In [2]:
from rcognita.data_buffers.data_buffer import DataBuffer
from rcognita.data_buffers.samplers import RollingSampler
from rcognita.system import InvertedPendulumPD
import numpy as np
import torch
import casadi as cs
from rcognita.model import ModelQuadLin, ModelQuadNoMix, Model
from rcognita.__utilities import rc
from typing import Union


data_buffer = DataBuffer(20)

/home/user/.pyenv/versions/3.10.9/envs/rcenv_dev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dim_inputs = 6
batch_size = 10

quad_matrix_source = np.random.randn(dim_inputs, dim_inputs)
linear_coefs_source = np.random.randn(1, dim_inputs)
inputs_source = np.random.randn(batch_size, dim_inputs)
inputs_1dim_source = np.random.randn(dim_inputs)

In [4]:
dtype = np.array

quad_matrix = dtype(quad_matrix_source)
linear_coefs = dtype(linear_coefs_source)
inputs = dtype(inputs_source)
inputs_1dim = dtype(inputs_source)[0, :]

In [5]:
class ModelQuadLin(Model):
    model_name = "ModelQuadLin"

    def __init__(
        self,
        quad_matrix_type: str,
        is_with_linear_terms=False,
        dim_inputs=None,
        weights=None,
        single_weight_min=1.0e-6,
        single_weight_max=1.0e3,
    ):
        assert (
            dim_inputs is not None or weights is not None
        ), "Need dim_inputs or weights"

        self.quad_matrix_type = quad_matrix_type
        self.is_with_linear_terms = is_with_linear_terms

        if weights is None:
            self._calculate_dims(dim_inputs)
            self.weight_min = single_weight_min * np.ones(self.dim_weights)
            self.weight_max = single_weight_max * np.ones(self.dim_weights)
            self.weights = (self.weight_min + self.weight_max) / 20.0
        else:
            self._calculate_dims(self._calculate_dim_inputs(len(weights)))
            assert self.dim_weights == len(weights), "Wrong shape of dim_weights"
            self.weights = weights

        self.update_and_cache_weights()

    def _calculate_dim_inputs(self, dim_weights):
        if self.quad_matrix_type == "diagonal":
            if self.is_with_linear_terms:
                return dim_weights // 2
            else:
                return dim_weights
        elif self.quad_matrix_type == "full":
            if self.is_with_linear_terms:
                return round((np.sqrt(1 + 4 * dim_weights) - 1) / 2)
            else:
                return round(np.sqrt(dim_weights))
        elif self.quad_matrix_type == "symmetric":
            if self.is_with_linear_terms:
                return round((np.sqrt(9 + 8 * dim_weights) - 1) / 2)
            else:
                return round((np.sqrt(1 + 8 * dim_weights) - 1) / 2)

    def _calculate_dims(self, dim_inputs):
        self.dim_inputs = dim_inputs
        self.dim_linear = dim_inputs if self.is_with_linear_terms else 0
        if self.quad_matrix_type == "diagonal":
            self.dim_quad = dim_inputs
        elif self.quad_matrix_type == "full":
            self.dim_quad = dim_inputs * dim_inputs
        elif self.quad_matrix_type == "symmetric":
            self.dim_quad = dim_inputs * (dim_inputs + 1) // 2

        self.dim_weights = self.dim_quad + self.dim_linear

    def cast_to_inputs_type(self, value, inputs):
        if isinstance(inputs, torch.Tensor):
            device = inputs.device
            return torch.FloatTensor(value).to(device)

        return value

    def forward_symmetric(self, inputs, weights):
        quad_matrix = ModelQuadLin.quad_matrix_from_flat_weights(
            weights[: self.dim_quad]
        )
        linear_coefs = (
            weights[None, self.dim_quad :] if self.is_with_linear_terms else None
        )

        return ModelQuadLin.quadratic_linear_form(
            inputs,
            self.cast_to_inputs_type(quad_matrix, inputs),
            self.cast_to_inputs_type(linear_coefs, inputs),
        )

    def forward_diagonal(self, inputs, weights):
        quad_matrix = np.diag(weights[: self.dim_quad])
        linear_coefs = (
            weights[None, self.dim_quad :] if self.is_with_linear_terms else None
        )

        return ModelQuadLin.quadratic_linear_form(
            inputs,
            self.cast_to_inputs_type(quad_matrix, inputs),
            self.cast_to_inputs_type(linear_coefs, inputs),
        )

    def forward_full(self, inputs, weights):
        quad_matrix = weights[: self.dim_quad].reshape(self.dim_inputs, self.dim_inputs)
        linear_coefs = (
            weights[None, self.dim_quad :] if self.is_with_linear_terms else None
        )

        return ModelQuadLin.quadratic_linear_form(
            inputs,
            self.cast_to_inputs_type(quad_matrix, inputs),
            self.cast_to_inputs_type(linear_coefs, inputs),
        )

    def forward(self, inputs, weights=None):
        if weights is None:
            weights = self.weights
        if self.quad_matrix_type == "symmetric":
            return self.forward_symmetric(inputs, weights)
        elif self.quad_matrix_type == "diagonal":
            return self.forward_diagonal(inputs, weights)
        elif self.quad_matrix_type == "full":
            return self.forward_full(inputs, weights)

    @staticmethod
    def quad_matrix_from_flat_weights(
        flat_weights: Union[np.array, cs.DM, torch.Tensor], tol=1e-7
    ):
        len_flat_weights = flat_weights.shape[0]
        dim_quad_matrix_float = (np.sqrt(1 + 8 * len_flat_weights) - 1) / 2
        dim_quad_matrix = round(dim_quad_matrix_float)
        assert np.isclose(
            dim_quad_matrix_float, dim_quad_matrix, tol
        ), f"Can't build quad matrix with flat_weights of dim {len_flat_weights}"

        quad_matrix = rc.zeros(
            (dim_quad_matrix, dim_quad_matrix), prototype=flat_weights
        )
        left_ids, right_ids = np.triu_indices(dim_quad_matrix)
        for weigth_idx, (i, j) in enumerate(zip(left_ids, right_ids)):
            quad_matrix[i, j] = flat_weights[weigth_idx]

        return quad_matrix

    @staticmethod
    def quadratic_linear_form(inputs, quad_matrix, linear_coefs=None):
        initial_dim_inputs = len(inputs.shape)
        assert (
            initial_dim_inputs == 1 or initial_dim_inputs == 2
        ), "Wrong shape of inputs can be 1d or 2d. Got {}".format(initial_dim_inputs)

        if initial_dim_inputs == 1:
            inputs = inputs.reshape(1, -1)
        assert (
            len(quad_matrix.shape) == 2
        ), "Wrong shape of quad matrix. Should be 2d. Got{}".format(
            len(quad_matrix.shape)
        )
        assert (
            quad_matrix.shape[0] == quad_matrix.shape[1]
        ), "Quad matrix should be square"
        assert (
            quad_matrix.shape[0] == inputs.shape[1]
        ), "Quad matrix should have same number of rows as inputs"

        quadratic_term = inputs @ quad_matrix @ inputs.T
        if len(quadratic_term.shape) > 0:
            quadratic_term = rc.diag(quadratic_term)

        if linear_coefs is not None:
            assert (
                len(linear_coefs.shape) == 2 and linear_coefs.shape[0] == 1
            ), "Wrong shape of linear coefs. Should be (1,n). Got {}".format(
                linear_coefs.shape
            )

            assert (
                quad_matrix.shape[1] == linear_coefs.shape[1]
            ), "Quad matrix should have same number of columns as linear coefs"

            linear_term = inputs @ linear_coefs.T
            output = quadratic_term + linear_term
        else:
            output = quadratic_term

        if initial_dim_inputs == 1:
            output = output.reshape(-1)
        return output

In [6]:
# ModelQuadLin(quad_matrix_type="full", dim_inputs=10, is_with_linear_terms=True).forward(
#     torch.ones(10).float().to("cuda:0")
# )
var = cs.MX.sym("x", (3, 10))
model = ModelQuadLin(quad_matrix_type="full", dim_inputs=10, is_with_linear_terms=True)
inferred = model(var)

fun = rc.to_casadi_function(inferred, var)

In [13]:
model(torch.ones((3, 5)).float(), torch.ones((3, 5)).float())

TypeError: ModelQuadLin.forward() got multiple values for argument 'weights'

In [11]:
model.weights

array([50.00000005, 50.00000005, 50.00000005, 50.00000005, 50.00000005,
       50.00000005, 50.00000005, 50.00000005, 50.00000005, 50.00000005,
       50.00000005, 50.00000005, 50.00000005, 50.00000005, 50.00000005,
       50.00000005, 50.00000005, 50.00000005, 50.00000005, 50.00000005,
       50.00000005, 50.00000005, 50.00000005, 50.00000005, 50.00000005,
       50.00000005, 50.00000005, 50.00000005, 50.00000005, 50.00000005,
       50.00000005, 50.00000005, 50.00000005, 50.00000005, 50.00000005,
       50.00000005, 50.00000005, 50.00000005, 50.00000005, 50.00000005,
       50.00000005, 50.00000005, 50.00000005, 50.00000005, 50.00000005,
       50.00000005, 50.00000005, 50.00000005, 50.00000005, 50.00000005,
       50.00000005, 50.00000005, 50.00000005, 50.00000005, 50.00000005,
       50.00000005, 50.00000005, 50.00000005, 50.00000005, 50.00000005,
       50.00000005, 50.00000005, 50.00000005, 50.00000005, 50.00000005,
       50.00000005, 50.00000005, 50.00000005, 50.00000005, 50.00

In [13]:
fun(np.ones((3, 10)))

DM([5500, 5500, 5500])

In [175]:
quadratic_linear_form(inputs, quad_matrix, linear_coefs)

array([[-12.19375124],
       [ 11.48625062],
       [ -1.1589163 ],
       [ -4.70181312],
       [ -1.34230182],
       [  2.18544219],
       [  2.66085847],
       [ -0.33756305],
       [-12.53730914],
       [ -2.84967403]])

In [176]:
quadratic_linear_form(inputs_1dim, quad_matrix, linear_coefs)

array([-12.19375124])

In [79]:
quad_matrix_from_flat_weights(cs.DM(np.random.randn(6)))

DM(
[[0.479169, -0.60329, -2.34685], 
 [0, -2.02354, -1.20931], 
 [0, 0, 1.41944]])

In [56]:
quad_matrix_from_flat_weights(np.arange(6))

array([[-0.89828672,  0.17927228,  0.10315711],
       [ 0.        , -1.06295708,  0.27266104],
       [ 0.        ,  0.        , -0.81095218]])

In [47]:
import numpy as np

quad_matrix = rc.zeros((3, 3), rc_type=rc.CASADI)
left_ids, right_ids = np.triu_indices(3)
for weigth_idx, (i, j) in enumerate(zip(left_ids, right_ids)):
    quad_matrix[i, j] = weights[weigth_idx]

0 0
0 1
0 2
1 1
1 2
2 2


In [45]:
quad_matrix

DM(
[[-0.898287, 0.179272, 0.103157], 
 [0, -1.06296, 0.272661], 
 [0, 0, -0.810952]])

In [38]:
quad_matrix[[2, 1], [2, 1]]

DM(
[[0, 0], 
 [0, 0]])

In [30]:
quad_matrix

array([[0., 1., 2.],
       [0., 3., 4.],
       [0., 0., 5.]])

In [26]:
np.triu_indices(3)

(array([0, 0, 0, 1, 1, 2]), array([0, 1, 2, 1, 2, 2]))

In [11]:
rc.outer(x, x)

DM(
[[4.53883, -1.15349, 2.21474], 
 [-1.15349, 0.293145, -0.562849], 
 [2.21474, -0.562849, 1.08069]])

In [10]:
result

DM(-7.07441)

In [7]:
polynom

DM([0.730796, -0.677986, -0.787233, 0.628992, 0.730344, 0.848028, -0.854866, 0.79309, 0.920884])

In [69]:
cs.diag(cs.DM(x) @ cs.DM(weights) @ cs.DM(x.T)).full()

NotImplementedError: Wrong number or type of arguments for overloaded function 'new_DM'.
  Possible prototypes are:
    DM()
    DM(Sparsity)
    DM(float)
    DM([[float]])
    DM(DM)
    DM([SXElem])
    DM(SX)
    DM(int,int)
    DM(Sparsity,DM)
  You have: '(Tensor)'


In [66]:
np.diag(x @ weights @ x.T).reshape(-1, 1)

array([[2.20434664],
       [3.23446814],
       [1.50832892],
       [0.14954387],
       [1.02458255],
       [1.92327833],
       [0.38708603],
       [2.88263956],
       [0.64751441],
       [0.43413925]])

In [12]:
for i in range(100):
    data_buffer.push_to_end(
        action=np.array([i, i, i], dtype=float),
        observation=np.array([i, i, i], dtype=float),
    )

In [16]:
model = ModelQuadNoMix(dim_input=3)

for batch in data_buffer.iter_batches(
    batch_size=4, mode="backward", keys=["action", "observation"], dtype=cs.DM
):
    print(batch)
    break

model(batch["observation"])

{'action': DM(
[[96, 96, 96], 
 [97, 97, 97], 
 [98, 98, 98], 
 [99, 99, 99]]), 'observation': DM(
[[96, 96, 96], 
 [97, 97, 97], 
 [98, 98, 98], 
 [99, 99, 99]])}


RuntimeError: .../casadi/core/matrix_impl.hpp:2000: Assertion "x.size()==y.size()" failed:
dot: Dimension mismatch

In [3]:
np.isnan([1, 2])

array([False, False])

In [3]:
data_buffer.to_pandas()

,observation,action
0,"[-80, -80, -80]","[80, 80, 80]"
1,"[-81, -81, -81]","[81, 81, 81]"
2,"[-82, -82, -82]","[82, 82, 82]"
3,"[-83, -83, -83]","[83, 83, 83]"
4,"[-84, -84, -84]","[84, 84, 84]"
5,"[-85, -85, -85]","[85, 85, 85]"
6,"[-86, -86, -86]","[86, 86, 86]"
7,"[-87, -87, -87]","[87, 87, 87]"
8,"[-88, -88, -88]","[88, 88, 88]"
9,"[-89, -89, -89]","[89, 89, 89]"


In [3]:
data_buffer.push_to_end(jopa=2)

In [11]:
data_buffer.push_to_end(jopa3=10)

In [22]:
data_buffer.to_pandas()

,observation,action,jopa,jopa2,jopa3,jopa4
0,"[-83, -83, -83]","[83, 83, 83]",nan,nan,nan,nan
1,"[-84, -84, -84]","[84, 84, 84]",nan,nan,nan,nan
2,"[-85, -85, -85]","[85, 85, 85]",nan,nan,nan,nan
3,"[-86, -86, -86]","[86, 86, 86]",nan,nan,nan,nan
4,"[-87, -87, -87]","[87, 87, 87]",nan,nan,nan,nan
5,"[-88, -88, -88]","[88, 88, 88]",nan,nan,nan,nan
6,"[-89, -89, -89]","[89, 89, 89]",nan,nan,nan,nan
7,"[-90, -90, -90]","[90, 90, 90]",nan,nan,nan,nan
8,"[-91, -91, -91]","[91, 91, 91]",nan,nan,nan,nan
9,"[-92, -92, -92]","[92, 92, 92]",nan,nan,nan,nan


In [21]:
data_buffer.push_to_end(observation=[1, 2, 3])

In [8]:
data_buffer[-1]["observation"]

{'observation': array([-9223372036854775808, -9223372036854775808, -9223372036854775808]),
 'action': array([-9223372036854775808, -9223372036854775808, -9223372036854775808]),
 'jopa': [2]}

In [18]:
np.isnan(np.full_like(np.array([10, 1]), fill_value=np.nan, dtype=float)[0])

True

In [19]:
np.full_like(np.array([10, 1]), fill_value=np.nan, dtype=float)

array([nan, nan])